In [1]:
#evansdata
#gather time stamps --camera and wf--
import numpy as np
import pandas as pd
import h5py
from datapipe import find_nearest, getData, rowsToColumns
from pathlib import Path
import pickle
import scipy.io as scio
from sklearn.externals import joblib

In [ ]:
#should respond to audio
basePath = './data/'
behaveVidPath = basePath +'AV2_'
mePath = basePath+ 'AV_2_'
images = np.load(basePath+'/AV_2.npy', mmap_mode='r')

In [2]:
#load in the models 
#eyes
#ears
#feet
#
#load in the videos


In [3]:
videoTS = np.ndarray.flatten(scio.loadmat(behaveVidPath+'camera_ts.mat')['cameraTime'])
wf_ts = np.ndarray.flatten(scio.loadmat(behaveVidPath+'wf_ts.mat')['cropped_wf'])
audStim = scio.loadmat(behaveVidPath+'audStim.mat')['audStim'][0]
visStim = scio.loadmat(behaveVidPath+'visStim.mat')['visStim'][0]
lickTime = scio.loadmat(behaveVidPath+'lickTime.mat')['lickTime'][0]
walkCropped = scio.loadmat(behaveVidPath+'walkCropped.mat')['walkTot'][0]
stimTot = np.sort(np.append(audStim, visStim))
stimType = np.zeros(len(stimTot))

In [8]:
hit, miss, FA, CR         = [],[],[],[]
responseT, lastStimT    = [],[] 
vidPeri, vidCurr          = [],[]
wheelPeri, wheelCurr      = [],[]
tifIndex, stimT           = [],[]

for vidInd, vidTime in enumerate(wf_ts):
    windowT = vidTime
    if((vidTime > videoTS[0]) and (vidTime+3.85< videoTS[-1])):
        stimInd = find_nearest(stimTot, windowT)
        stimTime = stimTot[stimInd]
        currIdx = find_nearest(videoTS, windowT)
        pupilPeri.append(getData(pupilCropped, 30, .2, (windowT-0.3), videoTS))
        pupilCurr.append(pupilCropped[currIdx])
        whiskPeri.append(getData(whiskCropped, 30, .2, (windowT-0.3), videoTS))
        whiskCurr.append(whiskCropped[currIdx])
        wheelPeri.append(getData(walkCropped, 30, .2, (windowT-0.3), videoTS))
        wheelCurr.append(walkCropped[currIdx])
        vidPeri.append(getData(me3, 30, .2, (windowT-0.3), videoTS))
        vidCurr.append(me3[currIdx])
        snoutPeri.append(getData(me1, 30, .2, (windowT-0.3), videoTS))
        snoutCurr.append(me1[currIdx])
        jawPeri.append(getData(me2, 30, .2, (windowT-0.3), videoTS))
        jawCurr.append(me2[currIdx])
        #todo if time is before the closest stim look at stim before 
        if stimTime > windowT: 
            if stimInd == -1:
                lastStimT.append(0)
                #get licktime to see if there is a response
                stimT.append(-1)
                hit.append(0)
                miss.append(0)
                FA.append(0)
                CR.append(0)
                responseT.append(0)
            else:
                stimInd -= 1
                lastStimT.append(windowT-stimTot[stimInd])
                #get licktime to see if there is a response
                stimT.append(stimType[stimInd])
                prev = hit[-1]
                hit.append(prev)
                prev = miss[-1]
                miss.append(prev)
                prev = FA[-1]
                FA.append(prev)
                prev = CR[-1]
                CR.append(prev)
                prev = responseT[-1]
                responseT.append(prev)
        else:
            lastStimT.append(windowT-stimTot[stimInd])
            stimT.append(stimType[stimInd])
            lickT = find_nearest(lickTime, windowT)
            if stimType[stimInd] ==1 and stimTime-lickT < 1.6:
                hit.append(1)
                miss.append(0)
                FA.append(0)
                CR.append(0)
                responseT.append(stimTime-lickT)
            elif stimType[stimInd] ==0 and stimTime-lickT < 1.6:
                hit.append(0)
                miss.append(0)
                FA.append(1)
                CR.append(0)
                responseT.append(stimTime-lickT)
            elif stimType[stimInd] ==1:
                hit.append(0)
                miss.append(1)
                FA.append(0)
                CR.append(0)
                responseT.append(0)
            else:
                hit.append(0)
                miss.append(0)
                FA.append(0)
                CR.append(1)
                responseT.append(0)
                
        tifIndex.append(find_nearest(wf_ts, windowT))

In [9]:
tifIndexNP = images[tifIndex]

size = len(wheelPeri)
inter = np.concatenate(([stimT], [pupilPeri],[pupilCurr],[snoutPeri],[snoutCurr],[jawPeri],[jawCurr], [wheelPeri],[wheelCurr], [responseT], [hit], [miss], [FA], [CR],
                                 [whiskPeri],[whiskCurr], [lastStimT], [vidPeri],[vidCurr]), axis=0)
if not len(inter) == 0: 
    inter = inter.reshape(-1, size)
else:
    print("skip KEY")
masdf = pd.DataFrame()
columns=["stimType","pupilPeri", "pupilCurr", "snoutPeri", "snoutCurr","jawPeri","jawCurr","wheelPeri","wheelCurr","responseT", "H", "M", "FA", "CR",
                 "whiskPeri", "whiskCurr", "lastStimT", "vidPeri", "vidCurr"]
for it, item in enumerate(columns):
    masdf[item] = inter[it]
# print(masdf)
dff = open("./data/masdfAV_2.pkl", "wb")
# dyf = open("./dfy/AV_2.pkl", "wb")
pickle.dump(masdf, dff)
np.save("./data/AV_2.npy", tifIndexNP)